# Calculate clear-sky layer

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# - - - - - - - - - - - - - - - - - - - - 
# DEFAULT IMPORTS - IN ALL NOTEBOKS
from src import configs

prjconf = configs.ProjectConfigParser()

# - - - - - - - - - - - - - - - - - - - - 
# NOTEBOOK SPECIFIC IMPORTS
import numpy as np
from pathlib import Path
import pandas as pd
import rasterio
from tqdm import tqdm
import nasa_hls

Get scenes for which create the layers.

In [2]:
# print(prjconf.get_scene_collection_names())

scene_collection_name = 'scenecoll_tile-32UNU_year-2018_product-L30_maxcc-75_minsc-0'
scene_collection = prjconf.read_scene_collection(scene_collection_name)

In [3]:
dir_tiffs = prjconf.get_path("Interim", "hls")
scene_collection["dir_tiff"] = [str(Path(dir_tiffs) / sid) for sid in scene_collection["sid"]]
scene_collection.head()

,sid,product,tile,date_Yj,date,cloud_cover,spatial_coverage,dir_tiff
0,HLS.L30.T32UNU.2018044.v1.4,L30,T32UNU,2018044,2018-02-13,35.0,28.0,/home/ben/Devel/Projects/classify-hls/data/int...
1,HLS.L30.T32UNU.2018259.v1.4,L30,T32UNU,2018259,2018-09-16,3.0,100.0,/home/ben/Devel/Projects/classify-hls/data/int...
2,HLS.L30.T32UNU.2018316.v1.4,L30,T32UNU,2018316,2018-11-12,61.0,28.0,/home/ben/Devel/Projects/classify-hls/data/int...
3,HLS.L30.T32UNU.2018188.v1.4,L30,T32UNU,2018188,2018-07-07,53.0,26.0,/home/ben/Devel/Projects/classify-hls/data/int...
4,HLS.L30.T32UNU.2018218.v1.4,L30,T32UNU,2018218,2018-08-06,55.0,39.0,/home/ben/Devel/Projects/classify-hls/data/int...


## Create clear-sky masks from the QA layers

More details on the conversion from QA-layer to clear sky masks can be found in [nasa_hls’s documentation - Create a clear-sky mask from the QA layer](https://benmack.github.io/nasa_hls/build/html/tutorials/Working_with_HLS_datasets_and_nasa_hls.html#Create-a-clear-sky-mask-from-the-QA-layer).

In [4]:
# valid ids from the link above
valid = [  0,   4,  16,  20,  32,  36,  48,  52,  64,  68,  80,  84,  96,
         100, 112, 116, 128, 132, 144, 148, 160, 164, 176, 180, 192, 196,
         208, 212, 224, 228, 240, 244]

In [5]:
#scene_dirs = list(prjconf.get_path("Interim", "hls").glob("*L30*32UNU*"))
for i, scenedir in tqdm(enumerate(scene_collection["dir_tiff"].values), total=len(scene_collection)):
    qa_path = list(Path(scenedir).rglob("*QA*"))[0]
    clear_path = Path(str(qa_path).replace("__QA", "__CLEAR"))
    clear = nasa_hls.hls_qa_layer_to_mask(qa_path,
                                              qa_valid=valid,
                                              keep_255=True,
                                              mask_path=clear_path,
                                              overwrite=False)
    assert clear == 0
# 100%|██████████| 24/24 [00:04<00:00,  5.27it/s]

100%|██████████| 24/24 [00:04<00:00,  5.27it/s]
